In [ ]:
"""TIENES QUE DOCUMENTAR ESTE SCRIPT,( PRUEBA CON 5 CULTIVOS ) RECUERDA QUE ESTE PRIMER ESCRIP TIENE COMO RESULTADO 

-UsaR MIS datos reales (demanda, beneficio, superficie)

CalculaR qué cultivar cada mes y en qué cantidad

Maximiza automáticamente tu beneficio total anual"""

In [ ]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus, value

# Cargar datasets
cultivos_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/cultivos_hortalizas_final.csv")
demanda_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/demanda_clientes.csv")
terreno_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/terreno_suelo_final.csv")

# Calcular columna de rendimiento kg/m2
cultivos_df["Rendimiento_kg_m2"] = cultivos_df["Rendimiento_promedio (kg/ha)"] / 10000

# Calcular demanda anual y precio medio por producto
demanda_resumen = demanda_df.groupby("Producto").agg(
    demanda_total_kg=("Kg_comprados", "sum"),
    precio_medio=("Precio_kg_€", "mean")
).reset_index()

coste_generico = 0.30
demanda_resumen["beneficio_kg"] = demanda_resumen["precio_medio"] - coste_generico

# Selección de productos
top_productos = demanda_resumen.sort_values(by="demanda_total_kg", ascending=False).head(5)
productos = top_productos["Producto"].tolist()
meses = list(range(1, 13))

# Superficie total en m2
superficie_total_m2 = terreno_df["Superficie_ha"].sum() * 10_000

# Diccionarios
beneficios = dict(zip(top_productos["Producto"], top_productos["beneficio_kg"]))
demandas = dict(zip(top_productos["Producto"], top_productos["demanda_total_kg"]))
rendimientos = dict(zip(cultivos_df["Nombre_cultivo"], cultivos_df["Rendimiento_kg_m2"]))

# Crear modelo
modelo = LpProblem("Maximizar_Beneficio_Anual", LpMaximize)
x = {(p, m): LpVariable(f"x_{p}_{m}", lowBound=0) for p in productos for m in meses}

# Objetivo
modelo += lpSum(x[p, m] * beneficios[p] for p in productos for m in meses)

# Funciones de restricciones
def restriccion_demanda(modelo, x, productos, meses, demandas):
    for p in productos:
        modelo += lpSum(x[p, m] for m in meses) <= demandas[p], f"DemandaMax_{p}"

def restriccion_superficie(modelo, x, productos, meses, rendimientos, superficie_total_m2):
    for m in meses:
        modelo += lpSum(x[p, m] * (1 / rendimientos.get(p, 1)) for p in productos) <= superficie_total_m2, f"TerrenoMes_{m}"

# Aplicar restricciones
restriccion_demanda(modelo, x, productos, meses, demandas)
restriccion_superficie(modelo, x, productos, meses, rendimientos, superficie_total_m2)

# Resolver
modelo.solve()

# Resultados
resultado = pd.DataFrame([
    {"Producto": p, "Mes": m, "Cantidad_kg": x[p, m].varValue}
    for p in productos for m in meses if x[p, m].varValue > 0
])

print("📌 Estado del modelo:", LpStatus[modelo.status])
print("💰 Beneficio total anual optimizado: €", round(value(modelo.objective), 2))
print("\n📅 Calendario óptimo de siembra:")
print(resultado)


In [ ]:
"""TIENES QUE DOCUMENTAR ESTE SCRIPT, ( PRUEBA CON 5 CULTIVOS )ESTE SEGUNDO SCRIP implementar la lógica de rotación en el modelo
Vamos a añadir restricciones para que:

-Si se siembra un cultivo en el mes "M"
-No se pueda volver a sembrar en los meses "M" + 1 y "M" + 2,....,+ DURACION - 1
-Para cada cultivo, la duración de la siembra es de 3 meses

Esto requiere generar restricciones cruzadas entre los meses. Lo haremos ahora en código usando 
solo estas 5 variables y sin siembra mínima.

RESULTADO CON ROTACIÓN
📌 Estado: Optimal

💰 Beneficio anual total: € 9,531.65

📅 Calendario válido con las restricciones de rotación aplicadas correctamente
(por ejemplo: no se repite el mismo cultivo en meses consecutivos según su duración)

🔎 ¿Qué significa que el beneficio sea el mismo?
Significa que el modelo:

Encontró una solución igual de rentable

Pero respetando las nuevas reglas de rotación
Esto sugiere que los cultivos han sido colocados estratégicamente en meses no solapados, cumpliendo la lógica.


"""

In [ ]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus, value

# Cargar datasets
cultivos_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/cultivos_hortalizas_final.csv")
demanda_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/demanda_clientes.csv")
terreno_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/terreno_suelo_final.csv")

# Calcular columna de rendimiento por m2
cultivos_df["Rendimiento_kg_m2"] = cultivos_df["Rendimiento_promedio (kg/ha)"] / 10000

# Calcular demanda y beneficio
demanda_resumen = demanda_df.groupby("Producto").agg(
    demanda_total_kg=("Kg_comprados", "sum"),
    precio_medio=("Precio_kg_€", "mean")
).reset_index()

coste_generico = 0.30
demanda_resumen["beneficio_kg"] = demanda_resumen["precio_medio"] - coste_generico

# Seleccionar top 5 productos
top_productos = demanda_resumen.sort_values(by="demanda_total_kg", ascending=False).head(5)
productos = top_productos["Producto"].tolist()
meses = list(range(1, 13))

# Beneficios y demandas
beneficios = dict(zip(top_productos["Producto"], top_productos["beneficio_kg"]))
demandas = dict(zip(top_productos["Producto"], top_productos["demanda_total_kg"]))

# Superficie total disponible mensual
superficie_total_m2 = terreno_df["Superficie_ha"].sum() * 10000

# Duración en meses
duracion_dias = cultivos_df.set_index("Nombre_cultivo")["Duración_cultivo_días"]
duracion_meses = np.ceil(duracion_dias / 30).astype(int)
duraciones = duracion_meses.loc[duracion_meses.index.intersection(productos)].to_dict()

# Rendimientos kg/m2
rendimientos = cultivos_df.set_index("Nombre_cultivo")["Rendimiento_kg_m2"].to_dict()

# Crear modelo
modelo = LpProblem("Maximizar_Beneficio_Con_Rotacion", LpMaximize)

# Variables de decisión: cantidad sembrada en kg
x = {
    (p, m): LpVariable(f"x_{p}_{m}", lowBound=0)
    for p in productos for m in meses
}

# Función objetivo
modelo += lpSum(x[p, m] * beneficios[p] for p in productos for m in meses)

# Restricción 1: demanda máxima
for p in productos:
    modelo += lpSum(x[p, m] for m in meses) <= demandas[p], f"DemandaMax_{p}"

# Restricción 2: superficie mensual disponible (convertimos kg a m2 usando rendimiento)
for m in meses:
    modelo += lpSum(x[p, m] * (1 / rendimientos[p]) for p in productos) <= superficie_total_m2, f"TerrenoMes_{m}"

# Restricción 3: evitar solapamientos por rotación
for p in productos:
    dur = duraciones[p]
    for m in meses:
        for k in range(1, dur):
            if m + k in meses:
                modelo += x[p, m] + x[p, m + k] <= superficie_total_m2, f"Rotacion_{p}_{m}_{m+k}"

# Resolver modelo
modelo.solve()

# Mostrar resultados
resultado = pd.DataFrame([
    {"Producto": p, "Mes": m, "Cantidad_kg": x[p, m].varValue}
    for p in productos for m in meses if x[p, m].varValue and x[p, m].varValue > 0
])

print("📌 Estado del modelo:", LpStatus[modelo.status])
print("💰 Beneficio total anual optimizado con rotación: €", round(value(modelo.objective), 2))
print("\n📅 Calendario óptimo con rotación:")
print(resultado)

# Opcional: exportar
# resultado.to_excel("data/calendario_optimo_con_rotacion.xlsx", index=False)


In [ ]:
"""DOCUMENTAR ESTE SCRIPT, ( PRUEBA CON TODOS CULTIVOS )ESTE TERCER SCRIP implementa la lógica de rotación en el modelo

-El estado del modelo

-El beneficio total anual óptimo

-El calendario completo de siembras mes a mes para todos los productos

-Puedes exportarlo a Excel fácilmente con la última línea





"""

In [ ]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus, value

# Cargar datasets
cultivos_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/cultivos_hortalizas_final.csv")
demanda_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/demanda_clientes.csv")
terreno_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/terreno_suelo_final.csv")

# Crear nueva columna de rendimiento en kg/m2
cultivos_df["Rendimiento_kg_m2"] = cultivos_df["Rendimiento_promedio (kg/ha)"] / 10000

# Calcular demanda total y beneficio por kg
demanda_resumen = demanda_df.groupby("Producto").agg(
    demanda_total_kg=("Kg_comprados", "sum"),
    precio_medio=("Precio_kg_€", "mean")
).reset_index()

coste_generico = 0.30
demanda_resumen["beneficio_kg"] = demanda_resumen["precio_medio"] - coste_generico

# Productos disponibles
productos_completos = demanda_resumen["Producto"].tolist()
meses = list(range(1, 13))

# Diccionarios
beneficios_all = dict(zip(demanda_resumen["Producto"], demanda_resumen["beneficio_kg"]))
demandas_all = dict(zip(demanda_resumen["Producto"], demanda_resumen["demanda_total_kg"]))

# Superficie total disponible mensual (convertida a m²)
superficie_total_m2 = terreno_df["Superficie_ha"].sum() * 10000

# Duración de cada cultivo en meses
duracion_dias = cultivos_df.set_index("Nombre_cultivo")["Duración_cultivo_días"]
duracion_meses = np.ceil(duracion_dias / 30).astype(int)
duraciones_all = duracion_meses.loc[duracion_meses.index.intersection(productos_completos)].to_dict()

# Rendimiento por m²
rendimientos_all = cultivos_df.set_index("Nombre_cultivo")["Rendimiento_kg_m2"].to_dict()

# Crear modelo
modelo_full = LpProblem("Maximizar_Beneficio_Todos_Con_Rotacion", LpMaximize)

# Variables de decisión (kg por cultivo y mes)
x_full = {
    (p, m): LpVariable(f"x_{p}_{m}", lowBound=0)
    for p in productos_completos for m in meses
}

# Función objetivo
modelo_full += lpSum(x_full[p, m] * beneficios_all[p] for p in productos_completos for m in meses)

# Restricción 1: demanda máxima por producto
for p in productos_completos:
    modelo_full += lpSum(x_full[p, m] for m in meses) <= demandas_all[p], f"DemandaMax_{p}"

# Restricción 2: superficie mensual convertida a partir de kg usando rendimiento
for m in meses:
    modelo_full += lpSum(
        x_full[p, m] * (1 / rendimientos_all[p])
        for p in productos_completos if p in rendimientos_all
    ) <= superficie_total_m2, f"TerrenoMes_{m}"

# Restricción 3: evitar solapamiento de siembras (rotación)
for p in duraciones_all:
    d = duraciones_all[p]
    for m in meses:
        for k in range(1, d):
            if m + k in meses:
                modelo_full += x_full[p, m] + x_full[p, m + k] <= superficie_total_m2, f"Rotacion_{p}_{m}_{m+k}"

# Resolver el modelo
modelo_full.solve()

# Resultados en DataFrame
resultado = pd.DataFrame([
    {"Producto": p, "Mes": m, "Cantidad_kg": x_full[p, m].varValue}
    for p in productos_completos for m in meses
    if x_full[p, m].varValue and x_full[p, m].varValue > 0
])

# Mostrar resultados
print("📌 Estado del modelo:", LpStatus[modelo_full.status])
print("💰 Beneficio total anual optimizado con rotación: €", round(value(modelo_full.objective), 2))
print("\n📅 Calendario óptimo de siembra para todos los cultivos:")
print(resultado)

# Guardar si es necesario
# resultado.to_excel("data/calendario_optimo_todos_con_rotacion.xlsx", index=False)


In [ ]:
"""🚀 ¿Qué significa esto?
Tienes ya un motor de planificación agrícola inteligente que:

Maximiza la rentabilidad total

Respeta límites de terreno

Evita plantar el mismo cultivo antes de tiempo

Funciona a escala real con tus datos"""

In [ ]:
"""
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
"""

In [ ]:
""" PRUEVA RESTRICCIONES CON 5 CULTIVOS"""

In [ ]:
"""
Importación de librerías y carga de datos
    ¿Qué hace?

            -Importa las librerías necesarias (pandas, numpy y pulp) para el análisis 
             y optimización.
            -Carga los tres datasets esenciales del proyecto: cultivos, demanda y terreno.
            -Prepara los datos iniciales, incluyendo:
                        <Cálculo del rendimiento en kg/m² para cada cultivo.
                        <Cálculo del beneficio por kg (precio medio - coste unitario).
                        <Extracción de los productos más demandados (top 5–10).

    ¿Para qué sirve?
        Establece el entorno de trabajo y transforma los datos brutos en variables clave 
        que usaremos en la optimización.
        
    """

In [35]:
#1-Importaciones y carga de datos
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus, value

# Cargar datasets
cultivos_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/cultivos_hortalizas_final.csv")
demanda_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/demanda_clientes.csv")
terreno_df = pd.read_csv("/Users/kardiahq/Desktop/33.final_proyect/agro/data/terreno_suelo_final.csv")

# Calcular columna de rendimiento por m²
cultivos_df["Rendimiento_kg_m2"] = cultivos_df["Rendimiento_promedio (kg/ha)"] / 10_000

# Agregar beneficio por kg
coste_generico = 0.30
demanda_resumen = demanda_df.groupby("Producto").agg(
    demanda_total_kg=("Kg_comprados", "sum"),
    precio_medio=("Precio_kg_€", "mean")
).reset_index()
demanda_resumen["beneficio_kg"] = demanda_resumen["precio_medio"] - coste_generico



In [36]:
#Selección de productos y parámetros
productos = demanda_resumen.sort_values(by="demanda_total_kg", ascending=False).head(5)["Producto"].tolist()
beneficios = dict(zip(demanda_resumen["Producto"], demanda_resumen["beneficio_kg"]))
demandas = dict(zip(demanda_resumen["Producto"], demanda_resumen["demanda_total_kg"]))
rendimientos = dict(zip(cultivos_df["Nombre_cultivo"], cultivos_df["Rendimiento_kg_m2"]))

duracion_dias = cultivos_df.set_index("Nombre_cultivo")["Duración_cultivo_días"]
duracion_meses = np.ceil(duracion_dias / 30).astype(int)
duraciones = duracion_meses.loc[duracion_meses.index.intersection(productos)].to_dict()

superficie_total_m2 = terreno_df["Superficie_ha"].sum() * 10_000
meses = list(range(1, 13))


In [ ]:
"""
Inicialización del modelo
    ¿Qué hace?
                -Crea el modelo de optimización lineal con PuLP.
                -Define las variables de decisión: cuántos kg sembrar por cultivo y por mes.

    ¿Para qué sirve?
        Establece la estructura básica del modelo matemático que vamos a optimizar.
        
"""

In [37]:
#Crear modelo y variables
modelo = LpProblem("Maximizar_Beneficio_Agricola", LpMaximize)
x = {(p, m): LpVariable(f"x_{p}_{m}", lowBound=0) for p in productos for m in meses}


In [38]:
#Función objetivo
modelo += lpSum(x[p, m] * beneficios[p] for p in productos for m in meses), "BeneficioTotal"

In [ ]:
"""
Restricción 1 – Límite por demanda máxima
    ¿Qué hace?
        -Asegura que no se siembren más kg de un producto que lo que se demanda en total.

    ¿Para qué sirve?
        -Evita la sobreproducción de cultivos que el mercado no va a absorber. 
        -Controla la cantidad total de producción por producto."""

In [40]:
# Restricción de demanda máxima
def restriccion_demanda(modelo, x, demandas, productos, meses):
    for p in productos:
        modelo += lpSum(x[p, m] for m in meses) <= demandas[p], f"DemandaMax_{p}"

# Llamada a la función pasando todos los argumentos necesarios
restriccion_demanda(modelo, x, demandas, productos, meses)

In [ ]:
"""
 Restricción 2 – Límite de superficie mensual
    ¿Qué hace?
            -Asegura que la superficie disponible por mes no se sobrepase.
            -Ajusta la cantidad sembrada por el rendimiento del cultivo (kg por m²).

    ¿Para qué sirve?
            -Hace coherente la relación entre la producción en kg y el uso real de superficie en m², 
             evitando errores de interpretación entre unidades.
             
"""

In [42]:
# Restricción de superficie mensual con ajuste por rendimiento
def restriccion_superficie(modelo, x, rendimientos, productos, meses, superficie_total_m2):
    for m in meses:
        modelo += lpSum(x[p, m] * (1 / rendimientos[p]) for p in productos) <= superficie_total_m2, f"SuperficieMes_{m}"

# Llamada a la función con los argumentos correctos
restriccion_superficie(modelo, x, rendimientos, productos, meses, superficie_total_m2)


In [ ]:
"""
Restricción 3 – Evitar siembra simultánea del mismo cultivo en meses consecutivos
    ¿Qué hace?
            -Implementa una lógica de “rotación básica” donde no se permite que el mismo 
             cultivo se siembre dos veces seguidas en su periodo de duración.

    ¿Para qué sirve?
            -Evita solapamientos poco realistas en los calendarios de cultivo. Refleja una 
              planificación lógica del uso del terreno.
              
"""

In [44]:
# Restricción de rotación de cultivos: evita solapamiento en los meses de duración del cultivo
def restriccion_rotacion(modelo, x, productos, duraciones, meses, superficie_total_m2):
    for p in productos:
        d = duraciones.get(p, 1)
        for m in meses:
            for k in range(1, d):
                if m + k in meses:
                    modelo += x[p, m] + x[p, m + k] <= superficie_total_m2, f"Rotacion_{p}_{m}_{m+k}"

# Llamada a la función
restriccion_rotacion(modelo, x, productos, duraciones, meses, superficie_total_m2)


In [ ]:
"""
Resolución del modelo
    ¿Qué hace?
            -Ejecuta el modelo de optimización con todas las restricciones anteriores.
            -Imprime el estado del modelo (si es óptimo) y el beneficio anual estimado.

    ¿Para qué sirve?
            -Obtiene la solución óptima (cuánto sembrar de cada cultivo, en qué mes) 
            maximizando el beneficio, sujeto a todas las condiciones del sistema.

"""

In [45]:
# Resolver el modelo
modelo.solve()

print("📌 Estado del modelo:", LpStatus[modelo.status])
print("💰 Beneficio total anual optimizado: €", round(value(modelo.objective), 2))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kardiahq/Desktop/33.final_proyect/agro/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0p/7n5ngc6d6r7flqnyyr2rm3vh0000gn/T/eeba86600c1c4f818f7dd4ab05596524-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/0p/7n5ngc6d6r7flqnyyr2rm3vh0000gn/T/eeba86600c1c4f818f7dd4ab05596524-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 233 COLUMNS
At line 836 RHS
At line 1065 BOUNDS
At line 1066 ENDATA
Problem MODEL has 228 rows, 60 columns and 542 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-228) rows, 0 (-60) columns and 0 (-542) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 9531.6542
After Postsolve, objective 9531.6542, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 9531.654157 - 0 iteration

In [ ]:
"""
Resultados
    ¿Qué hace?
            -Extrae los valores óptimos de las variables de decisión.
            -Genera un dataframe con el calendario de siembras y lo muestra de forma clara.

    ¿Para qué sirve?
            -Visualiza el plan agrícola resultante, mostrando qué cultivos sembrar en qué meses y en qué cantidades.

"""

In [46]:
# Mostrar resultados finales
resultado = pd.DataFrame([
    {"Producto": p, "Mes": m, "Cantidad_kg": x[p, m].varValue}
    for p in productos for m in meses if x[p, m].varValue and x[p, m].varValue > 0
])

print("\n📅 Calendario óptimo:")
print(resultado)


📅 Calendario óptimo:
     Producto  Mes  Cantidad_kg
0   Zanahoria    1       3250.3
1  Col rizada    1       3006.5
2    Pimiento    1       2842.5
3         Ajo    9       2737.9
4    Guisante    9       2364.6


In [ ]:
"""
Ese resultado indica que el modelo está funcionando correctamente con las tres restricciones 
aplicadas:

✅ Demanda máxima (no se siembra más de lo que se demanda por producto).

✅ Superficie mensual (no se excede el área total disponible, ahora ajustada con los rendimientos).

✅ Rotación de cultivos (evita solapamientos de siembras durante la duración de cada cultivo).

"""

In [ ]:
"""
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------        

"""